In [1]:
# imports
import json
import os
from typing import List, Dict
import math

In [2]:
# file dir
filename = 'saved_conversations_recbot_gpt4o.jsonl'
# get the current working directory
ROOT_DIR = os.path.dirname(os.getcwd())

In [3]:
def read_jsonl(fpath: str) -> List[Dict]:
    res = []
    with open(fpath, 'r') as f:
        for line in f:
            data = json.loads(line)
            res.append(data)
    return res

In [4]:
def compute_dcg(rank):
    return 1 / math.log2(rank + 1)

In [5]:
# load file
data = read_jsonl(os.path.join(ROOT_DIR, 'data', filename))
print(data[0])

{'context': 'canned shrimp', 'target': ['Original Pouch Tuna'], 'answer': [19402880, 20385438, 18648443, 28082975, 20503651, 20530756, 20383661, 18648459, 20992209, 20220093, 18648017, 17497901, 41411745, 27242960, 18649386, 19758284, 19750614, 19942135, 6633, 7272], 'LLM_ranker_placement': [], 'Rank_tool_placement': [18], 'NDCG@20': 0.0, 'Baseline': 0.2354}


In [6]:
# iterate through data and compute dcg by only considering the top converted item from each query
llm_dcg = []
llm_mean_placement = []
base_dcg = []
base_mean_placement = []
for d in data:
    rank = d['LLM_ranker_placement']
    base = d['Rank_tool_placement']
    try:
        llm_dcg.append(compute_dcg(rank[0]))
        llm_mean_placement.append(rank[0])
    except:
        llm_dcg.append(0)
        llm_mean_placement.append(20)
    
    base_dcg.append(compute_dcg(base[0]))
    base_mean_placement.append(base[0])

# compute the average dcg
llm_avg = sum(llm_dcg) / len(llm_dcg)
base_avg = sum(base_dcg) / len(base_dcg)
# print the results
print(f'LLM DCG: {llm_avg:.4f}')
print(f'Base DCG: {base_avg:.4f}')
# compute the average placement
llm_avg_placement = sum(llm_mean_placement) / len(llm_mean_placement)
base_avg_placement = sum(base_mean_placement) / len(base_mean_placement)
# print the results
print(f'LLM Mean Placement: {llm_avg_placement:.4f}')
print(f'Base Mean Placement: {base_avg_placement:.4f}')

LLM DCG: 0.5338
Base DCG: 0.6860
LLM Mean Placement: 7.1486
Base Mean Placement: 4.9557
